<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#proc_df" data-toc-modified-id="proc_df-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>proc_df</a></span></li></ul></div>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
%matplotlib inline

from fastai.imports import *
from fastai.torch_imports import *
from fastai.dataset import *
from fastai.learner import *
from fastai.structured import *
from fastai.column_data import *
import feather
import gc

In [3]:
df_raw = feather.read_dataframe('all_train_cont')
# df_test = feather.read_dataframe('all_test_cont')

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [4]:
dep = 'target'

In [5]:
for df in [df_raw]:
    df.drop('price', axis=1, inplace=True)
    for f in ['purchase_date']:
        df[f] = df[f].astype(np.int64) * 1e-9
    for c in ['avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12', 
              'purchase_amount_sum_ratio', 'installments_ratio',
              'month_diff_ratio',
              'purchase_amount_sum_ratio']:
        df[c] = df[c].replace(np.inf, df[c].median())

In [6]:
train_cards = df_raw['card_id'].unique()

In [7]:
train_cards.shape

(201917,)

In [8]:
val_cards = np.random.choice(train_cards, 40000)

In [9]:
val_cards.shape

(40000,)

In [10]:
val_idx = df_raw.index[df_raw.card_id.isin(val_cards)]

In [11]:
val_cards2 = np.random.choice(val_cards, 8000)

In [12]:
df_valid = df_raw.iloc[val_idx]

In [13]:
val_idx2 = df_valid.index[df_valid.card_id.isin(val_cards2)]

In [14]:
df_raw.shape, df_valid.shape, len(val_idx2)

((19249698, 91), (3433782, 91), 678088)

In [15]:
df_valid_trn = df_valid[~df_valid.card_id.isin(val_cards2)]

In [16]:
df_valid_trn.shape, df_valid.shape, len(val_idx2)

((2755694, 91), (3433782, 91), 678088)

In [17]:
gc.collect()

7

In [18]:
val_idx.shape

(3433782,)

In [16]:
trn_idx = df_raw[~df_raw.card_id.isin(val_cards)]

MemoryError: 

In [19]:
val_idx.shape, trn_idx.shape, df_raw.shape

NameError: name 'trn_idx' is not defined

In [18]:
df_raw.columns.values

array(['authorized_flag', 'card_id', 'installments', 'category_3', 'merchant_category_id', 'merchant_id',
       'month_lag', 'purchase_amount', 'purchase_date', 'purchase_Year', 'purchase_Month', 'purchase_Week',
       'purchase_Day', 'purchase_Dayofweek', 'purchase_Dayofyear', 'purchase_Is_month_end',
       'purchase_Is_month_start', 'purchase_Is_quarter_end', 'purchase_Is_quarter_start',
       'purchase_Is_year_end', 'purchase_Is_year_start', 'purchase_Hour', 'purchase_Minute',
       'purchase_Second', 'purchase_Elapsed', 'purchased_on_weekend', 'purchased_on_weekday', 'month_diff',
       'purchase_date_successive_diff', 'Christmas_Day_2017', 'Mothers_Day_2017', 'fathers_day_2017',
       'Children_day_2017', 'Valentine_Day_2017', 'Black_Friday_2017', 'Mothers_Day_2018', 'duration',
       'amount_month_ratio', 'merchant_address_id', 'numerical_range', 'merchant_rating',
       'merchant_group_id', 'merchant_category_id_y', 'subsector_id', 'numerical_1', 'numerical_2',
       '

In [57]:
pd.set_option('display.max_rows', 100)

In [13]:
df_raw.dtypes

authorized_flag                  category
card_id                            object
installments                      float64
category_3                       category
merchant_category_id             category
merchant_id                        object
month_lag                           int64
purchase_amount                   float64
purchase_date                     float64
purchase_Year                    category
purchase_Month                   category
purchase_Week                    category
purchase_Day                     category
purchase_Dayofweek               category
purchase_Dayofyear               category
purchase_Is_month_end                bool
purchase_Is_month_start              bool
purchase_Is_quarter_end              bool
purchase_Is_quarter_start            bool
purchase_Is_year_end                 bool
purchase_Is_year_start               bool
purchase_Hour                    category
purchase_Minute                  category
purchase_Second                  c

In [19]:
df_raw.select_dtypes(include=['category']).columns.values

array(['authorized_flag', 'category_3', 'merchant_category_id', 'purchase_Year', 'purchase_Month',
       'purchase_Week', 'purchase_Day', 'purchase_Dayofweek', 'purchase_Dayofyear', 'purchase_Hour',
       'purchase_Minute', 'purchase_Second', 'purchased_on_weekend', 'purchased_on_weekday',
       'Christmas_Day_2017', 'Mothers_Day_2017', 'fathers_day_2017', 'Children_day_2017',
       'Valentine_Day_2017', 'Black_Friday_2017', 'Mothers_Day_2018', 'merchant_rating', 'merchant_group_id',
       'subsector_id', 'category_1', 'most_recent_sales_range', 'most_recent_purchases_range',
       'active_months_lag3', 'active_months_lag6', 'active_months_lag12', 'category_4', 'city_id',
       'state_id', 'category_2', 'days_feature1', 'days_feature1_ratio', 'days_feature2',
       'days_feature2_ratio', 'days_feature3', 'days_feature3_ratio', 'feature_1', 'feature_2', 'feature_3',
       'feature_max', 'feature_mean', 'feature_min', 'feature_sum', 'feature_var', 'hist_first_buy',
       'hist_

In [8]:
df_test.select_dtypes(include=['category']).columns.values

array(['authorized_flag', 'category_3', 'merchant_category_id', 'purchase_Year', 'purchase_Month',
       'purchase_Week', 'purchase_Day', 'purchase_Dayofweek', 'purchase_Dayofyear', 'purchase_Hour',
       'purchase_Minute', 'purchase_Second', 'purchased_on_weekend', 'purchased_on_weekday',
       'Christmas_Day_2017', 'Mothers_Day_2017', 'fathers_day_2017', 'Children_day_2017',
       'Valentine_Day_2017', 'Black_Friday_2017', 'Mothers_Day_2018', 'merchant_rating', 'merchant_group_id',
       'subsector_id', 'category_1', 'most_recent_sales_range', 'most_recent_purchases_range',
       'active_months_lag3', 'active_months_lag6', 'active_months_lag12', 'category_4', 'city_id',
       'state_id', 'category_2', 'days_feature1', 'days_feature1_ratio', 'days_feature2',
       'days_feature2_ratio', 'days_feature3', 'days_feature3_ratio', 'feature_1', 'feature_2', 'feature_3',
       'feature_max', 'feature_mean', 'feature_min', 'feature_sum', 'feature_var', 'hist_first_buy',
       'hist_

In [19]:
cat_flds = df_raw.select_dtypes(include=['category']).columns.values

In [20]:
cat_flds = cat_flds.tolist() + ['card_id', 'merchant_id', 'merchant_address_id']

In [21]:
gc.collect()

0

In [29]:
pd.set_option('display.max_rows', 100)

In [23]:
df_raw.head().T

0  \
authorized_flag                                          1   
card_id                                    C_ID_da2090f28e   
installments                                             0   
category_3                                               0   
merchant_category_id                                   623   
merchant_id                                M_ID_f001319a61   
month_lag                                              -11   
purchase_amount                                         40   
purchase_date                                  1.48323e+09   
purchase_Year                                         2017   
purchase_Month                                           1   
purchase_Week                                           52   
purchase_Day                                             1   
purchase_Dayofweek                                       6   
purchase_Dayofyear                                       1   
purchase_Is_month_end                                False   
purchase_Is_month_start                               True   
purchase_Is_quarter_end                              False   
purchase_Is_quarter_start                             True   
purchase_Is_year_end                                 False   
purchase_Is_year_start                                True   
purchase_Hour                                            0   
purchase_Minute                                          0   
purchase_Second                                          8   
purchase_Elapsed                                1483228808   
purchased_on_weekend                                     1   
purchased_on_weekday                                     0   
month_diff                                              14   
purchase_date_successive_diff                            0   
Christmas_Day_2017                                       0   
Mothers_Day_2017                                         0   
fathers_day_2017                                         0   
Children_day_2017                                        0   
Valentine_Day_2017                                       0   
Black_Friday_2017                                        0   
Mothers_Day_2018                                         0   
duration                                               560   
amount_month_ratio                                 2.85714   
merchant_address_id            M_ID_f001319a61_623_4_0_9_1   
numerical_range                                          0   
merchant_rating                                         12   
merchant_group_id                                     2119   
merchant_category_id_y                                 623   
subsector_id                                             4   
numerical_1                                     -0.0574706   
numerical_2                                     -0.0574706   
category_1                                               N   
most_recent_sales_range                                  B   
most_recent_purchases_range                              B   
avg_sales_lag3                                        1.02   
avg_purchases_lag3                                 1.02696   
active_months_lag3                                       3   
avg_sales_lag6                                           1   
avg_purchases_lag6                                 1.01143   
active_months_lag6                                       6   
avg_sales_lag12                                       1.07   
avg_purchases_lag12                                1.06246   
active_months_lag12                                     12   
category_4                                               N   
city_id                                                 -1   
state_id                                                 9   
category_2                                               1   
CLV                                                 843.02   
CLV_old                                             216088   
CLV_ratio                  

In [14]:
df_raw.shape, df_test.shape

((19249698, 91), (11825701, 91))

In [22]:
bool_flds = df_raw.select_dtypes(include=['bool']).columns.values

In [23]:
cat_flds = cat_flds + bool_flds.tolist()

In [24]:
cont_flds = [n for n in df_raw.columns if n not in cat_flds and n!= 'outliers']

In [25]:
gc.collect()

0

In [15]:
# %%time
# df_raw = df_raw[cat_flds+cont_flds]
# df, y, nas, mapper_t = proc_df(df_raw, 'target', do_scale=True)

In [26]:
for c in bool_flds:
    df_valid[c] = df_valid[c].astype('category')

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
[n for n in df_valid.drop(cat_flds,axis=1).columns if not is_numeric_dtype(df_valid[n])]

[]

In [28]:
for c in cat_flds: df_valid[c] = df_valid[c].astype('category')

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
gc.collect()

0

In [ ]:
%%time
df_raw = df_raw[cat_flds+cont_flds+[dep]]
df, y, nas, mapper_t = proc_df(df_raw, 'target', do_scale=True)

In [31]:
df_valid_trn.shape, df_valid.shape, len(val_idx2)

((2772190, 91), (3450901, 91), 678711)

In [32]:
y_val = df_valid['target']

### proc_df

In [30]:
df_sorted = df_valid.sort_values(['card_id', 'purchase_date'])

In [31]:
df_sorted.reset_index(inplace=True)

In [32]:
df_sorted.drop('index', axis=1, inplace=True)

In [34]:
df_valid.shape, df_sorted.shape

((3433782, 91), (3433782, 91))

In [35]:
%%time
df,y, nas, mapper_t = proc_df(df_sorted, 'target', do_scale=True)

CPU times: user 1min, sys: 3.28 s, total: 1min 4s
Wall time: 20.3 s


In [36]:
gc.collect()

27

In [27]:
%%time
df_test_copy = df_test[cat_flds+cont_flds+[dep]].copy()
df_t, _, nas, mapper = proc_df(df_test_copy,'target', do_scale=True, mapper=mapper_t)

NameError: name 'mapper_t' is not defined

In [37]:
df.shape

(3433782, 91)

In [38]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

In [39]:
emb_c = {n: len(c.cat.categories)+1 for n,c in df_valid[cat_flds].items()}
emb_c

{'authorized_flag': 3,
 'category_3': 4,
 'merchant_category_id': 328,
 'purchase_Year': 3,
 'purchase_Month': 13,
 'purchase_Week': 53,
 'purchase_Day': 32,
 'purchase_Dayofweek': 8,
 'purchase_Dayofyear': 366,
 'purchase_Hour': 25,
 'purchase_Minute': 61,
 'purchase_Second': 61,
 'purchased_on_weekend': 3,
 'purchased_on_weekday': 3,
 'Christmas_Day_2017': 101,
 'Mothers_Day_2017': 101,
 'fathers_day_2017': 101,
 'Children_day_2017': 101,
 'Valentine_Day_2017': 101,
 'Black_Friday_2017': 101,
 'Mothers_Day_2018': 29,
 'merchant_rating': 17,
 'merchant_group_id': 109392,
 'subsector_id': 42,
 'category_1': 3,
 'most_recent_sales_range': 6,
 'most_recent_purchases_range': 6,
 'active_months_lag3': 4,
 'active_months_lag6': 7,
 'active_months_lag12': 13,
 'category_4': 3,
 'city_id': 272,
 'state_id': 26,
 'category_2': 6,
 'days_feature1': 644,
 'days_feature1_ratio': 647,
 'days_feature2': 488,
 'days_feature2_ratio': 505,
 'days_feature3': 233,
 'days_feature3_ratio': 234,
 'feature_

In [40]:
emb_szs = [(c, min(30, (c+1)//2)) for _,c in emb_c.items()]
metrics=[rmse]

In [41]:
y_range=(np.min(y)*1,np.max(y))

In [42]:
emb_szs

[(3, 2),
 (4, 2),
 (328, 30),
 (3, 2),
 (13, 7),
 (53, 27),
 (32, 16),
 (8, 4),
 (366, 30),
 (25, 13),
 (61, 30),
 (61, 30),
 (3, 2),
 (3, 2),
 (101, 30),
 (101, 30),
 (101, 30),
 (101, 30),
 (101, 30),
 (101, 30),
 (29, 15),
 (17, 9),
 (109392, 30),
 (42, 21),
 (3, 2),
 (6, 3),
 (6, 3),
 (4, 2),
 (7, 4),
 (13, 7),
 (3, 2),
 (272, 30),
 (26, 13),
 (6, 3),
 (644, 30),
 (647, 30),
 (488, 30),
 (505, 30),
 (233, 30),
 (234, 30),
 (6, 3),
 (4, 2),
 (3, 2),
 (6, 3),
 (8, 4),
 (3, 2),
 (8, 4),
 (10, 5),
 (2005, 30),
 (1955, 30),
 (1963, 30),
 (1991, 30),
 (233, 30),
 (5, 3),
 (36248, 30),
 (207210, 30),
 (207210, 30),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2),
 (3, 2)]

In [43]:
len(cat_flds), len(cont_flds), len(df.columns)

(63, 27, 91)

In [45]:
df.shape

(3433782, 91)

In [47]:
df[643378:643395]

authorized_flag  card_id  installments  category_3  \
643378                2     6769      0.295405           2   
643379                2     6769      1.999314           3   
643380                2     6769      0.295405           2   
643381                2     6769      0.295405           2   
643382                1     6769      0.295405           2   
643383                2     6769      0.295405           2   
643384                2     6769      0.295405           2   
643385                2     6769      0.295405           2   
643386                2     6769     -0.556550           1   
643387                2     6769      0.295405           2   
643388                2     6769      1.147359           3   
643389                2     6769     -0.556550           1   
643390                2     6769      1.147359           3   
643391                2     6769      0.295405           2   
643392                2     6769      0.295405           2   
643393                2     6769      1.147359           3   
643394                2     6769      0.295405           2   

        merchant_category_id  merchant_id  month_lag  purchase_amount  \
643378                   269       164944   1.091061        -0.000064   
643379                   321          526   1.091061        -0.002828   
643380                   263       183722   1.091061        -0.003234   
643381                    86        83008   1.091061        -0.003209   
643382                   282        55457   1.091061        -0.003539   
643383                   282        55457   1.091061        -0.003539   
643384                    28       169141   1.091061        -0.003311   
643385                   269       131376   1.091061        -0.001745   
643386                     8       115522   1.091061        -0.003281   
643387                   160       178958   1.091061        -0.003539   
643388                   196       171153   1.091061        -0.003135   
643389                   190       184010   1.357698        -0.002427   
643390                    28       198513   1.357698        -0.002533   
643391                   216        65693   1.357698        -0.003530   
643392                   134       101128   1.357698        -0.003227   
643393                   190       137752   1.357698        -0.002322   
643394                   102       180318   1.624336        -0.002657   

        purchase_date  purchase_Year  ...  hist_last_buy  installments_ratio  \
643378       0.767393              1  ...            195            0.026446   
643379       0.801695              1  ...            195            0.026446   
643380       0.802041              1  ...            195            0.026446   
643381       0.807492              1  ...            195            0.026446   
643382       0.811787              1  ...            195            0.026446   
643383       0.811813              1  ...            195            0.026446   
643384       0.833055              1  ...            195            0.026446   
643385       0.833306              1  ...            195            0.026446   
643386       0.842458              1  ...            195            0.026446   
643387       0.848663              1  ...            195            0.026446   
643388       0.851916              1  ...            195            0.026446   
643389       0.915228              2  ...            195            0.026446   
643390       0.916183              2  ...            195            0.026446   
643391       1.089687              2  ...            195            0.026446   
643392       1.096555              2  ...            195            0.026446   
643393       1.114606              2  ...            195            0.026446   
643394       1.150165              2  ...            195            0.026446   

        month_diff_ratio  new_first_buy  new_last_buy  observed_elapsed_time  \
643378         -0.749105            168           190  

In [48]:
val_idx2 = list(range(0, 643395))

In [49]:
len(val_idx2)

643395

In [50]:
gc.collect()

1874

In [54]:
len(val_idx2), df.shape, y.shape

(643395, (3433782, 91), (3433782,))

In [52]:
len(np.unique(df.columns.values))

91

In [53]:
md = ColumnarModelData.from_data_frame('./', val_idx2, df, y.astype(np.float32), cat_flds=cat_flds, bs=512, test_df=df)

In [55]:
%%time
m = md.get_learner(emb_szs, len(df.columns)-len(cat_flds),
                   0.4, 1, [1000, 500], [0.1, 0.01], y_range=y_range)

CPU times: user 1.85 s, sys: 306 ms, total: 2.15 s
Wall time: 2.52 s


In [56]:
m.lr_find()

 53%|█████▎    | 2904/5450 [00:50<00:43, 58.69it/s, loss=1.68] 

In [57]:
m.sched.plot()

In [58]:
lr=5e-5; wd=1e-6

In [60]:
%time m.fit(lr, 1, cycle_len=2, cycle_mult=2, metrics=[rmse])

epoch      trn_loss   val_loss   rmse                          
    0      0.75214    3.83553    1.869829  
    1      0.6297     3.637147   1.816519                      
CPU times: user 2min 48s, sys: 29.5 s, total: 3min 18s
Wall time: 3min 9s


[array([3.63715]), 1.8165192963266164]

In [61]:
l = m.predict()

In [62]:
l.shape

(643395, 1)

In [63]:
df_valid.shape

(3433782, 91)

In [64]:
len(val_idx)

3433782

In [66]:
l_t = m.predict(is_test=True)

In [72]:
l_t.shape, l.shape, df_valid.shape

((3433782, 1), (643395, 1), (3433782, 91))

In [75]:
df_valid['target_new'] = l_t

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [77]:
df_valid[['card_id', 'target', 'target_new']].head()

card_id    target  target_new
1   C_ID_efced389a0 -1.840332    0.658630
5   C_ID_ecf77791d8 -1.065921    1.004230
6   C_ID_a24df943dd  3.130923    0.989231
15  C_ID_ce778b86f8  1.162205    0.969341
18  C_ID_7ba7751d53  0.679358    0.706181

In [96]:
df_haa = df_valid.groupby('card_id').agg({'target': ['median'], 'target_new': ['median']})

In [97]:
df_haa.columns = ['_'.join(c) for c in df_haa.columns.values]

In [98]:
gc.collect()

28

In [99]:
rmse(df_haa.target_median.values, df_haa.target_new_median.values)

3.9617159691277544

In [100]:
df_haa.head()

target_median  target_new_median
card_id                                          
C_ID_0005b2f279       2.945304          -1.311934
C_ID_0005b5804f       0.685599          -1.000813
C_ID_0007a60a33      -1.614517          -1.164612
C_ID_000844d8f1      -0.868258          -1.182117
C_ID_000a6044e9      -3.774958          -1.102848

In [101]:
df_vla = df_sorted[:643395]

In [102]:
df_vla.shape, l.shape

((643395, 91), (643395, 1))

In [103]:
df_vla['target_pred'] = l

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [104]:
df_vla.head()

authorized_flag          card_id  installments category_3  \
0               1  C_ID_0005b2f279           1.0          1   
1               1  C_ID_0005b2f279           1.0          1   
2               1  C_ID_0005b2f279           1.0          1   
3               1  C_ID_0005b2f279           1.0          1   
4               1  C_ID_0005b2f279           1.0          1   

  merchant_category_id      merchant_id  month_lag  purchase_amount  \
0                  818  M_ID_431d369269        -11            106.0   
1                  278  M_ID_f064c2a380        -11             62.9   
2                  307  M_ID_256a900ace        -10            100.0   
3                  842  M_ID_30be6cf889         -7             38.0   
4                  278  M_ID_8c0ebe57c6         -7             30.0   

   purchase_date purchase_Year  ... installments_ratio month_diff_ratio  \
0   1.485686e+09          2017  ...           0.040816         1.001172   
1   1.485695e+09          2017  ...           0.040816         1.001172   
2   1.486204e+09          2017  ...           0.040816         1.001172   
3   1.494040e+09          2017  ...           0.040816         1.001172   
4   1.495985e+09          2017  ...           0.040816         1.001172   

  new_first_buy new_last_buy observed_elapsed_time outliers  \
0           NaN          NaN                   NaN        0   
1           NaN          NaN                   NaN        0   
2           NaN          NaN                   NaN        0   
3           NaN          NaN                   NaN        0   
4           NaN          NaN                   NaN        0   

  purchase_amount_sum_ratio quarter    target target_pred  
0                  0.071279     2.0  2.945304    0.658630  
1                  0.071279     2.0  2.945304    1.004230  
2                  0.071279     2.0  2.945304    0.989231  
3                  0.071279     2.0  2.945304    0.969341  
4                  0.071279     2.0  2.945304    0.706181  

[5 rows x 92 columns]

In [120]:
df_lol = df_vla.groupby('card_id').agg({'target': ['mean'], 'target_pred': ['mean']})

In [121]:
df_lol.columns = ['_'.join(c) for c in df_lol.columns.values]

In [122]:
rmse(df_lol.target_mean.values, df_lol.target_pred_mean.values)

nan

In [125]:
df_lol.tail()

target_mean  target_pred_mean
card_id                                       
C_ID_fffaed41f0          NaN               NaN
C_ID_fffb67f260          NaN               NaN
C_ID_fffd070e0e          NaN               NaN
C_ID_fffd24dcf3          NaN               NaN
C_ID_fffd93c8bf          NaN               NaN

In [138]:
df_vla[df_vla['card_id']=='C_ID_fffd93c8bf']

Empty DataFrame
Columns: [authorized_flag, card_id, installments, category_3, merchant_category_id, merchant_id, month_lag, purchase_amount, purchase_date, purchase_Year, purchase_Month, purchase_Week, purchase_Day, purchase_Dayofweek, purchase_Dayofyear, purchase_Is_month_end, purchase_Is_month_start, purchase_Is_quarter_end, purchase_Is_quarter_start, purchase_Is_year_end, purchase_Is_year_start, purchase_Hour, purchase_Minute, purchase_Second, purchase_Elapsed, purchased_on_weekend, purchased_on_weekday, month_diff, purchase_date_successive_diff, Christmas_Day_2017, Mothers_Day_2017, fathers_day_2017, Children_day_2017, Valentine_Day_2017, Black_Friday_2017, Mothers_Day_2018, duration, amount_month_ratio, merchant_address_id, numerical_range, merchant_rating, merchant_group_id, merchant_category_id_y, subsector_id, numerical_1, numerical_2, category_1, most_recent_sales_range, most_recent_purchases_range, avg_sales_lag3, avg_purchases_lag3, active_months_lag3, avg_sales_lag6, avg_purchases_lag6, active_months_lag6, avg_sales_lag12, avg_purchases_lag12, active_months_lag12, category_4, city_id, state_id, category_2, CLV, CLV_old, CLV_ratio, card_id_count_ratio, days_feature1, days_feature1_ratio, days_feature2, days_feature2_ratio, days_feature3, days_feature3_ratio, feature_1, feature_2, feature_3, feature_max, feature_mean, feature_min, feature_sum, feature_var, hist_first_buy, hist_last_buy, installments_ratio, month_diff_ratio, new_first_buy, new_last_buy, observed_elapsed_time, outliers, purchase_amount_sum_ratio, quarter, target, target_pred]
Index: []

[0 rows x 92 columns]

In [131]:
df_lol.target_mean.values

array([ 2.9453 ,  0.6856 , -1.61452, ...,      nan,      nan,      nan])

In [137]:
rmse(df_lol.target_mean.values, df_lol.target_pred_mean.values)

0.8951409180500303

In [135]:
df_lol.fillna(0, inplace=True)

In [139]:
df_vla['target'].isnull().mean()

0.0

In [140]:
df_vla.head()

authorized_flag          card_id  installments category_3  \
0               1  C_ID_0005b2f279           1.0          1   
1               1  C_ID_0005b2f279           1.0          1   
2               1  C_ID_0005b2f279           1.0          1   
3               1  C_ID_0005b2f279           1.0          1   
4               1  C_ID_0005b2f279           1.0          1   

  merchant_category_id      merchant_id  month_lag  purchase_amount  \
0                  818  M_ID_431d369269        -11            106.0   
1                  278  M_ID_f064c2a380        -11             62.9   
2                  307  M_ID_256a900ace        -10            100.0   
3                  842  M_ID_30be6cf889         -7             38.0   
4                  278  M_ID_8c0ebe57c6         -7             30.0   

   purchase_date purchase_Year  ... installments_ratio month_diff_ratio  \
0   1.485686e+09          2017  ...           0.040816         1.001172   
1   1.485695e+09          2017  ...           0.040816         1.001172   
2   1.486204e+09          2017  ...           0.040816         1.001172   
3   1.494040e+09          2017  ...           0.040816         1.001172   
4   1.495985e+09          2017  ...           0.040816         1.001172   

  new_first_buy new_last_buy observed_elapsed_time outliers  \
0           NaN          NaN                   NaN        0   
1           NaN          NaN                   NaN        0   
2           NaN          NaN                   NaN        0   
3           NaN          NaN                   NaN        0   
4           NaN          NaN                   NaN        0   

  purchase_amount_sum_ratio quarter    target target_pred  
0                  0.071279     2.0  2.945304    0.658630  
1                  0.071279     2.0  2.945304    1.004230  
2                  0.071279     2.0  2.945304    0.989231  
3                  0.071279     2.0  2.945304    0.969341  
4                  0.071279     2.0  2.945304    0.706181  

[5 rows x 92 columns]